In [1]:
import os

In [2]:
%pwd

'd:\\Project\\redwine_end-to-end\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Project\\redwine_end-to-end'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    train_data_path : Path
    test_data_path :Path
    model_name : str
    l1_ratio :float
    alpha : float
    target_column :str

In [9]:
from src.constants import *
from src.utils.common import read_yaml_file,create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        param_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml_file(config_filepath)
        self.params=read_yaml_file(param_filepath)
        self.schema=read_yaml_file(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_train_config(self)->ModelTrainerConfig:
        
        config=self.config.model_training
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_train_data=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            l1_ratio=params.l1_ratio,
            alpha=params.alpha,
            target_column=schema.name,
        )
        return model_train_data

In [ ]:
from sklearn.linear_model import ElasticNet
from src.logging import logger
import os
import pickle
import pandas as pd 

In [ ]:
class ModelTrainer:
    def __init__(self,config :ModelTrainerConfig):
        self.config=config
        
    def train_config(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)
        
        train_X=train_data.drop([self.config.target_column],axis=1)
        test_X=test_data.drop([self.config.target_column],axis=1)
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]
        
        ls=ElasticNet(l1_ratio=self.config.l1_ratio,alpha=self.config.alpha ,random_state=42)
        ls.fit(train_X,train_y)
        
        

        file_path = os.path.join(self.config.root_dir, self.config.model_name)

        # Open the file in write-binary mode
        with open(file_path, "wb") as f:
            pickle.dump(ls, f)


        

In [21]:
try:
    config=ConfigurationManager()
    model_train_config=config.get_model_train_config()
    model_train=ModelTrainer(config=model_train_config)
    model_train.train_config()
    
except Exception as e:
    raise e

[2025-02-19 23:01:07,089]: INFO: common:  yaml file config\config.yaml has loaded succesfully
[2025-02-19 23:01:07,093]: INFO: common:  yaml file params.yaml has loaded succesfully
[2025-02-19 23:01:07,097]: INFO: common:  yaml file schema.yaml has loaded succesfully
[2025-02-19 23:01:07,101]: INFO: common:  path has been created in artifacts
[2025-02-19 23:01:07,103]: INFO: common:  path has been created in artifacts/model_trainer
